In [1]:
from src.data.tushare.model.cne5 import TuShareCNE5_Calculator
cne5 = TuShareCNE5_Calculator()
cne5.Update('exposure')
cne5.Update('risk')


In [2]:
from src.data.tushare.download.daily import DailyMoneyFlow
DailyMoneyFlow().get_data(20241104)

,secid,buy_sm_vol,buy_sm_amount,sell_sm_vol,sell_sm_amount,buy_md_vol,buy_md_amount,sell_md_vol,sell_md_amount,buy_lg_vol,buy_lg_amount,sell_lg_vol,sell_lg_amount,buy_elg_vol,buy_elg_amount,sell_elg_vol,sell_elg_amount,net_mf_vol,net_mf_amount
0,1,206641,23472.72,214673,24397.62,294219,33423.61,300228,34133.57,349891,39772.04,351405,39933.79,280765,31919.03,265211,30122.43,-66602,-7439.96
1,2,557125,49903.84,383794,34410.17,636053,56992.08,573613,51429.29,496666,44509.93,574776,51486.75,247204,22159.22,404864,36238.86,-182446,-16153.23
2,4,75115,13725.40,64391,11822.04,45394,8307.64,52668,9630.07,26685,4909.84,24742,4527.00,3369,622.08,8762,1585.84,2299,428.43
3,6,286767,24512.43,197046,16857.81,292197,24981.22,317110,27118.47,230250,19685.66,277444,23679.77,96853,8279.79,114466,9803.05,-93641,-7936.06
4,7,47080,3313.73,54514,3838.73,52907,3730.79,44947,3173.40,29532,2082.85,30665,2159.18,2176,154.71,1569,110.77,27082,1912.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5088,688799,3813,1498.85,3640,1431.49,5319,2090.90,5359,2107.46,1958,770.76,2090,821.56,0,0.00,0,0.00,-763,-297.29
5089,688800,24307,8921.19,25042,9211.46,18769,6904.80,19942,7339.27,11353,4183.04,7961,2920.42,2903,1077.74,4387,1615.61,2069,773.56
5090,688819,29010,8197.77,28051,7928.01,20181,5708.49,19625,5551.70,8531,2411.08,10370,2932.08,992,282.18,668,187.73,-4120,-1163.00
5091,688981,72765,62502.24,84901,72982.84,181190,155670.85,183621,157781.59,215116,184889.83,220352,189364.82,107512,92388.61,87708,75322.27,12858,11539.99


In [2]:
import numpy as np
import pandas as pd

from typing import Literal

from src.factor.classes import StockFactorCalculator
from src.data import TSData

from src.func.transform import apply_ols
from src.func.singleton import singleton

def amihud(date , n_months : int , lag_months : int = 0):
    start_date , end_date = TSData.CALENDAR.td_start_end(date , n_months , 'm' , lag_months)
    ret = TSData.TRADE.get_returns(start_date , end_date , pivot=True)
    vol = TSData.TRADE.get_volumes(start_date , end_date , volume_type='volume' , pivot=True)
    amihud = (TSData.TRADE.mask_min_finite(ret).abs() / vol).mean() * (10 ** 6)
    return amihud

class illiq_1m(StockFactorCalculator):
    init_date = 20070101
    category0 = 'behavior'
    category1 = 'liquidity'
    description = '1个月Amihud非流动性因子'
    
    def calc_factor(self, date: int):
        return amihud(date , 1)

from cProfile import Profile
with Profile() as pr:
    v = illiq_1m().calc_factor(20241104) 
# pr.print_stats(1)
v

secid
1          0.075768
2          0.076334
4          2.393358
6          1.284054
7          2.196220
            ...    
920019          NaN
920066          NaN
920088          NaN
920099          NaN
920118    28.938717
Length: 5363, dtype: float64

In [1]:
from src.api.factor import factor_hierarchy
hier = factor_hierarchy()
hier.factor_df()




,level,file_name,factor_name,init_date,category0,category1,description
0,level0,correlation_beta,beta_12m,20070101,behavior,correlation,12个月贝塔
1,level0,correlation_beta,beta_1m,20070101,behavior,correlation,1个月贝塔
2,level0,correlation_beta,beta_2m,20070101,behavior,correlation,2个月贝塔
3,level0,correlation_beta,beta_3m,20070101,behavior,correlation,3个月贝塔
4,level0,correlation_beta,beta_6m,20070101,behavior,correlation,6个月贝塔
...,...,...,...,...,...,...,...
94,level0,volatility_skew,price_weiskew12m,20070101,behavior,volatility,12个月成交量加权偏度
95,level0,volatility_skew,price_weiskew1m,20070101,behavior,volatility,1个月成交量加权偏度
96,level0,volatility_skew,price_weiskew2m,20070101,behavior,volatility,2个月成交量加权偏度
97,level0,volatility_skew,price_weiskew3m,20070101,behavior,volatility,3个月成交量加权偏度


In [7]:
hier.update_jobs(file_name = 'liquidity_illiq')
df = hier.jobs()
df['level'].unique()

array([illiq_12m(from20070101,behavior,liquidity)[0dates],
       illiq_1m(from20070101,behavior,liquidity)[0dates],
       illiq_2m(from20070101,behavior,liquidity)[0dates],
       illiq_3m(from20070101,behavior,liquidity)[0dates],
       illiq_6m(from20070101,behavior,liquidity)[0dates]], dtype=object)

In [5]:
df = hier.jobs()
df['factor'].unique()


array([illiq_12m(from20070101,behavior,liquidity)[0dates],
       illiq_1m(from20070101,behavior,liquidity)[0dates],
       illiq_2m(from20070101,behavior,liquidity)[0dates],
       illiq_3m(from20070101,behavior,liquidity)[0dates],
       illiq_6m(from20070101,behavior,liquidity)[0dates]], dtype=object)

In [6]:
hier.Update()

Factor : illiq_12m at date 20070104 deploy successful
Factor : illiq_1m at date 20070104 deploy successful
Factor : illiq_2m at date 20070104 deploy successful
Factor : illiq_3m at date 20070104 deploy successful
Factor : illiq_6m at date 20070104 deploy successful
Factor : illiq_12m at date 20070105 deploy successful
Factor : illiq_1m at date 20070105 deploy successful
Factor : illiq_2m at date 20070105 deploy successful
Factor : illiq_3m at date 20070105 deploy successful
Factor : illiq_6m at date 20070105 deploy successful
Factor : illiq_12m at date 20070108 deploy successful
Factor : illiq_1m at date 20070108 deploy successful
Factor : illiq_2m at date 20070108 deploy successful
Factor : illiq_3m at date 20070108 deploy successful
Factor : illiq_6m at date 20070108 deploy successful
Factor : illiq_12m at date 20070109 deploy successful
Factor : illiq_1m at date 20070109 deploy successful
Factor : illiq_2m at date 20070109 deploy successful
Factor : illiq_3m at date 20070109 deploy 